<a href="https://colab.research.google.com/github/ctrivino1/ML_class_final/blob/main/Chris_Trivino_final_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Libraries

In [1]:
!pip install pyforest
!pip install git+https://github.com/keras-team/keras-preprocessing.git
!pip install keras-rectified-adam
!pip install tensorflow_addons
import tensorflow_addons as tfa
from keras_radam import RAdam
import pyforest
import shutil, os
from keras_preprocessing.image import ImageDataGenerator
import tensorflow as tf
from tensorflow import keras
from keras import layers
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.preprocessing import image_dataset_from_directory
from keras_radam import RAdam


  Cloning https://github.com/keras-team/keras-preprocessing.git to /tmp/pip-req-build-204ritti
  Running command git clone -q https://github.com/keras-team/keras-preprocessing.git /tmp/pip-req-build-204ritti


In [3]:
!git clone https://github.com/ctrivino1/ML_class_final.git


fatal: destination path 'ML_class_final' already exists and is not an empty directory.


In [4]:
training_dir = "/content/ML_class_final/train_imgs_1"
test_dir = "/content/ML_class_final/test_imgs_1"
df=pd.read_csv("/content/ML_class_final/train_1.csv")
df.label = df.label.astype(str)

<IPython.core.display.Javascript object>

In [5]:
train_datagen=ImageDataGenerator(rescale=1./255,shear_range=.2,zoom_range=.2,horizontal_flip=True,width_shift_range=0.05,height_shift_range=0.05,validation_split=.2,)


train_generator = train_datagen.flow_from_dataframe(dataframe=df, 
                                         directory=training_dir, 
                                         x_col="image_id", 
                                         y_col="label", 
                                         class_mode="categorical", 
                                         target_size=(224,224), 
                                         batch_size=64,
                                         subset="training",
                                         seed=69,
                                         )






Found 13694 validated image filenames belonging to 5 classes.


In [6]:

valid_generator = train_datagen.flow_from_dataframe(dataframe=df, 
                                         directory=training_dir, 
                                         x_col="image_id", 
                                         y_col="label", 
                                         class_mode="categorical", 
                                         target_size=(224,224), 
                                         batch_size=64,
                                         subset="validation",
                                         seed=69)

Found 3423 validated image filenames belonging to 5 classes.


In [8]:

resnet50 = tf.keras.applications.ResNet50(
    include_top=False, weights='imagenet', input_tensor=None,
    input_shape=(224,224,3), pooling="max"
)


vgg16 = tf.keras.applications.VGG16(
    include_top=False, weights='imagenet', input_tensor=None,
    input_shape=(224,224,3), pooling="max"
)
# Freeze the pretrained CNN's
resnet50.trainable = False
vgg16.trainable = False

# optimizers
radam = tfa.optimizers.RectifiedAdam(learning_rate=1e-3)

# Adds ResNet50 into a model with a output layers with 5 classifaction nodes
model = keras.Sequential([vgg16,
    tf.keras.layers.Dense(units=256, activation='LeakyReLU'),
    tf.keras.layers.Dense(units=256, activation='sigmoid'),
    tf.keras.layers.Dense(units=256, activation='LeakyReLU'),
    tf.keras.layers.Dense(5,activation="softmax") # number of perceptrons
])
model.compile(loss="categorical_crossentropy",
optimizer=radam, metrics=['acc'])
STEP_SIZE_TRAIN=train_generator.n//train_generator.batch_size
STEP_SIZE_VALID=valid_generator.n//valid_generator.batch_size
model.fit(train_generator,
          validation_data = valid_generator,
          steps_per_epoch = STEP_SIZE_TRAIN,
          validation_steps = STEP_SIZE_VALID,
          epochs=15)





Epoch 1/15
213/213 [==============================] - 272s 1s/step - loss: 1.1225 - acc: 0.5964 - val_loss: 0.9813 - val_acc: 0.6518
Epoch 2/15
213/213 [==============================] - 259s 1s/step - loss: 0.9307 - acc: 0.6596 - val_loss: 0.9391 - val_acc: 0.6704
Epoch 3/15
213/213 [==============================] - 260s 1s/step - loss: 0.8951 - acc: 0.6689 - val_loss: 0.9540 - val_acc: 0.6651
Epoch 4/15
213/213 [==============================] - 260s 1s/step - loss: 0.8812 - acc: 0.6789 - val_loss: 0.8636 - val_acc: 0.6837
Epoch 5/15
213/213 [==============================] - 260s 1s/step - loss: 0.8584 - acc: 0.6827 - val_loss: 0.8566 - val_acc: 0.6843
Epoch 6/15
213/213 [==============================] - 260s 1s/step - loss: 0.8542 - acc: 0.6886 - val_loss: 0.9539 - val_acc: 0.6300
Epoch 7/15
213/213 [==============================] - 260s 1s/step - loss: 0.8457 - acc: 0.6884 - val_loss: 0.8694 - val_acc: 0.6810
Epoch 8/15
213/213 [==============================] - 259s 1s/step - 

In [19]:
#downloading the model as a zipfile

import shutil
shutil.make_archive('70_perc_model', 'zip', '70_perc_model')

'/content/70_perc_model.zip'

In [25]:
import pathlib

def predict_an_image(model, file_path):
  # Load the image
  img = keras.preprocessing.image.load_img(file_path, target_size=(224, 224))
  # Get the image into the shape we need for our network
  img_array = keras.preprocessing.image.img_to_array(img)
  img_array = tf.expand_dims(img_array, 0) 
  # Predict the class
  predictions = model.predict(img_array)
  score = tf.nn.softmax(predictions[0])  
  return np.argmax(score)
# Loop through all images in our test directory and make
# a prediction
testdir = pathlib.Path('/content/ML_class_final/test_imgs_1')
image_paths = list(testdir.glob('*.jpg'))
for image_path in image_paths:
  prediction = predict_an_image(model, str(image_path))
  print(image_path, prediction)

/content/ML_class_final/test_imgs_1/4123594620.jpg 4


<IPython.core.display.Javascript object>

/content/ML_class_final/test_imgs_1/522535695.jpg 4


<IPython.core.display.Javascript object>

/content/ML_class_final/test_imgs_1/330459705.jpg 4


...

/content/ML_class_final/test_imgs_1/1774341872.jpg 4


<IPython.core.display.Javascript object>

/content/ML_class_final/test_imgs_1/1478823591.jpg 1


<IPython.core.display.Javascript object>

/content/ML_class_final/test_imgs_1/498277992.jpg 3


In [46]:
df = pd.DataFrame(columns = ['Filename'])

def get_row_prediction(row):
  return predict_an_image(model, '/content/ML_class_final/test_imgs_1' + row['Filename'])
df['predicted_class'] = df.apply(get_row_prediction, axis=1)

<IPython.core.display.Javascript object>

In [47]:
df.head()

,Filename,predicted_class


,image_id,class
